In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("news.csv")
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
data.drop("Unnamed: 0", axis=1, inplace=True)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   object
dtypes: object(3)
memory usage: 148.6+ KB


In [5]:
data["label"] = pd.get_dummies(data["label"], drop_first=True)

In [6]:
data.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


In [7]:
data["label"].value_counts()

1    3171
0    3164
Name: label, dtype: int64

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [9]:
X = cv.fit_transform(data["text"])

In [10]:
X.shape

(6335, 67659)

In [11]:
y = data["label"]

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X, y, test_size=0.25, random_state=10)

In [14]:
from xgboost import XGBClassifier
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [15]:
xgb.fit(Xtrain, ytrain)
ypred = xgb.predict(Xtest)

In [16]:
xgb.score(Xtest, ytest)

0.9330808080808081

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

In [18]:
confusion_matrix(ytest, ypred)

array([[713,  56],
       [ 50, 765]], dtype=int64)

In [19]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       769
           1       0.93      0.94      0.94       815

    accuracy                           0.93      1584
   macro avg       0.93      0.93      0.93      1584
weighted avg       0.93      0.93      0.93      1584

